In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas
import requests
from decimal import Decimal
from re import sub
from geopy.geocoders import Nominatim
import string
import time

import warnings
warnings.filterwarnings("ignore")

### Extract data from one prop

In [2]:
def extractPropertyInfo(URL):
    df = pd.DataFrame([])
    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0'}
    soup = BeautifulSoup(requests.get(URL,  headers=headers, verify=False).content, "html.parser")
    
    #2022 assement
    try:
        assess_2022 = soup.select_one("span[id=MainContent_lblGenAssessment]").text
        assess_2022 = Decimal(sub(r'[^\d.]', '', assess_2022))
        df = pd.concat([df, pd.DataFrame({'assess_2022':assess_2022}, index=[0])], axis=1)
    except:
        pass
    
    # 2021 
    try:
        table = soup.find('table', id='MainContent_grdHistoryValuesAsmt')
        row = table.find_all('tr')[1]
        columns = row.find_all('td')
        assess_2021 = columns[3].text.strip()
        assess_2021 = Decimal(sub(r'[^\d.]', '', assess_2021))
        df = pd.concat([df, pd.DataFrame({'assess_2021':assess_2021}, index=[0])], axis=1)
    except:
        pass
    
    # address
    try:
        addy = soup.select_one("span[id=MainContent_lblTab1Title]").text + ', Stamford, CT'
        df = pd.concat([df, pd.DataFrame({'address':addy}, index=[0])], axis=1)
        geolocator = Nominatim(user_agent='stamford_prop_tax')
        location = geolocator.geocode(addy)
        df = pd.concat([df, pd.DataFrame({'longitude':location.longitude, 'latitude':location.latitude}, index=[0])], axis=1)
    except:
        pass
    
    #sale price and date
    try:
        sale_price = soup.select_one("span[id=MainContent_lblPrice]").text
        sale_price = float(Decimal(sub(r'[^\d.]', '', sale_price)))
        sale_date = soup.select_one("span[id=MainContent_lblSaleDate]").text
        df = pd.concat([df, pd.DataFrame({'sale_price':sale_price,'sale_date':sale_date}, index=[0])], axis=1)
    except:
        pass
    
    #year built and sq ft and acreage
    try:
        year_built = float(soup.select_one("span[id=MainContent_ctl01_lblYearBuilt]").text)
        sq_ft = float(soup.select_one("span[id=MainContent_ctl01_lblBldArea]").text.replace(',',''))
        table = soup.find('table', id='MainContent_tblLand')
        row = table.find_all('tr')[0]
        columns = row.find_all('td')
        acres = float(columns[1].text.strip())
        df = pd.concat([df, pd.DataFrame({'year_built':year_built, 'sq_ft':sq_ft, 'acreage':acres}, index=[0])], axis=1)
    except:
        pass
    
    # attributes
    try:
        table = soup.find('table', id='MainContent_ctl01_grdCns')
        attributes = {}
        for row in table.find_all('tr'):    
            # Find all data for each column
            columns = row.find_all('td')
            if(columns != []):
                col_name = columns[0].text.strip().replace(':','')
                col_val = columns[1].text.strip().replace(':','')
                attributes[col_name] = col_val
        attributes = pd.DataFrame(data=attributes, index=[0]).replace('',np.nan)
        df = pd.concat([df,attributes], axis=1)
    except:
        pass
    
    # building areas
    try:
        table = soup.find('table', id='MainContent_ctl01_grdSub')
        building_areas = {}
        for row in table.find_all('tr'):    
            # Find all data for each column
            columns = row.find_all('td')
            if(columns != []):
                col_name = columns[0].text.strip().replace(':','')
                col_val = float(columns[2].text.strip().replace(',',''))
                building_areas[col_name] = col_val
        building_areas = pd.DataFrame(data=building_areas, index=[0]).replace('',np.nan)
        building_areas = building_areas.iloc[:,:-1]
        df = pd.concat([df,building_areas], axis=1)
    except:
        pass
    
    # extra 
    try:
        table = soup.find('table', id='MainContent_grdXf')
        extra_features = {}
        for row in table.find_all('tr'):    
            # Find all data for each column
            columns = row.find_all('td')
            if(columns != []):
                col_name = columns[0].text.strip().replace(':','')
                col_val = columns[2].text.strip().replace(' S.F','')
                col_val = col_val.replace(' UNITS', '')
                col_val = float(col_val)
                if col_name not in extra_features.keys():
                    extra_features[col_name] = col_val
                else:
                    extra_features[col_name] += col_val
        extra_features = pd.DataFrame(data=extra_features, index=[0]).replace('',np.nan)
        df = pd.concat([df,extra_features], axis=1)
    except:
        pass
    
    # outbuildings
    try:
        table = soup.find('table', id='MainContent_grdOb')
        outbuildings = {}
        for row in table.find_all('tr'):    
            # Find all data for each column
            columns = row.find_all('td')
            if(columns != []):
                col_name = columns[0].text.strip().replace(':','')
                col_val = float(columns[4].text.strip().replace(' S.F.','').replace(' UNITS', ''))
                if col_name not in outbuildings.keys():
                    outbuildings[col_name] = col_val
                else:
                    outbuildings[col_name] += col_val
        outbuildings = pd.DataFrame(data=outbuildings, index=[0]).replace('',np.nan)
        df = pd.concat([df,outbuildings], axis=1)
    except:
        pass
    
    # property type
    try:
        table = soup.find('table', id='MainContent_tblLandUse')
        row = table.find_all('tr')[1]
        columns = row.find_all('td')
        prop_type = columns[1].text.strip()
        df = pd.concat([df, pd.DataFrame({'prop_type':prop_type}, index=[0])], axis=1)
    except:
        pass
    
    return df

### Extract props from street

In [3]:
def extractStreet(URL):
    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0'}
    soup = BeautifulSoup(requests.get(URL,  headers=headers, verify=False).content, "html.parser")
    
    df_street = pd.DataFrame([])
    for a in soup.find_all('a', href=True):
        if 'Parcel' in a['href']:
            parcel_url = 'https://gis.vgsi.com/stamfordct/'+a['href']
            try:
                df_parcel = extractPropertyInfo(parcel_url)
                df_parcel = df_parcel.loc[:,~df_parcel.columns.duplicated()]
                df_street = pd.concat([df_street, df_parcel], axis=0)
            except:
                pass

    return df_street

### Extract street names from letter

In [4]:
def extractLetter(letter):
    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0'}
    URL = 'https://gis.vgsi.com/stamfordct/Streets.aspx?Letter=%s' % letter
    soup = BeautifulSoup(requests.get(URL,  headers=headers, verify=False).content, "html.parser")
    
    df_letter = pd.DataFrame([])
    for a in soup.find_all('a', href=True):
        if 'Streets.aspx?Name' in a['href']:
            print('Street: %s' % a['href'].replace('Streets.aspx?Name=',''))
            street_url = 'https://gis.vgsi.com/stamfordct/'+a['href']
            df_street = extractStreet(street_url)
            df_letter = pd.concat([df_letter, df_street], axis=0)

    return df_letter


### loop over all letters an build output file

In [ ]:
# lazy mans way to get list of letters
az_Upper = string.ascii_uppercase
letters = []
for i in az_Upper:
    letters.append(i)
    
# run the loop
t1 = time.time()
df_property = pd.DataFrame([])
for letter in letters:
    df_letter = extractLetter(letter)
    df_letter.to_csv('/Users/Ryan/Dropbox/Projects/StamfordPropTax_2022/ExtractedData/letter_%s.csv' % letter)
    df_property = pd.concat([df_property, df_letter],axis=0)  
t2 = time.time()

df_letter.to_csv('/Users/Ryan/Dropbox/Projects/StamfordPropTax_2022/ExtractedData/allProperties.csv')
print('Time: ' (t2-t1)/60/60)

Street: ABEL AVENUE
Street: ABERDEEN STREET
Street: ACOSTA STREET
Street: ACRE VIEW DRIVE
Street: ADAMS AVENUE
Street: AKBAR ROAD
Street: ALBERT PLACE
Street: ALBIN ROAD
Street: ALDEN STREET
Street: ALEXANDRA DRIVE
Street: ALFRED LANE
Street: ALGONQUIN AVENUE
Street: ALLISON ROAD
Street: ALMA ROCK ROAD
Street: ALPINE STREET
Street: ALTON ROAD
Street: ALVORD LANE
Street: AMELIA PLACE
Street: AMHERST COURT
Street: AMHERST PLACE
Street: ANDERSON STREET
Street: ANDOVER ROAD
Street: ANN STREET
Street: ANNIE PLACE
Street: ANTHONY STREET
Street: APPLE TREE DRIVE
Street: APPLE TREE LANE
Street: APPLE VALLEY ROAD
Street: APPLEBEE ROAD
Street: AQUILA ROAD
Street: ARBOR ROAD
Street: ARCHER LANE
Street: ARDEN LANE
Street: ARDMORE ROAD
Street: ARDSLEY ROAD
Street: ARLINGTON ROAD
Street: ARNOLD DRIVE
Street: ARROW HEAD DRIVE
Street: ARTHUR PLACE
Street: ASHTON ROAD
Street: ASPEN LANE
Street: ATLANTIC STREET
Street: AULDWOOD ROAD
Street: AUSTIN AVENUE
Street: AUTUMN LANE
Street: AVERY STREET
Street: 